# Getting Started with q-Kangaroo

**q-Kangaroo** is a symbolic computation engine for q-series, designed to replace
Garvan's Maple packages with a modern, high-performance Rust backend and a
Python API. It is aimed at researchers in number theory, combinatorics, and
partition theory.

This notebook walks you from installation through your first identity
verification, covering the core concepts step by step.

## Installation

Install q-Kangaroo from PyPI:

```bash
pip install q-kangaroo
```

Then import the package:

```python
from q_kangaroo import *
```

## Creating a Session

Every computation starts with a `QSession` that manages the symbolic
computation context -- variables, expressions, and the underlying
hash-consed arena.

In [1]:
from q_kangaroo import QSession

s = QSession()
print(s)

QSession(symbols: 0, expressions: 0)


## Your First q-Pochhammer Symbol

The **q-Pochhammer symbol** $(a;q)_n = \prod_{k=0}^{n-1}(1 - aq^k)$ is the
fundamental building block of q-series. The `aqprod` function computes it as
a formal power series.

The parameters are `aqprod(session, coeff_num, coeff_den, power, n, order)`,
where $a = \frac{\text{coeff\_num}}{\text{coeff\_den}} \cdot q^{\text{power}}$,
`n` is the product length, and `order` is the truncation order.

In [2]:
from q_kangaroo import aqprod

# Compute (q;q)_3 = (1-q)(1-q^2)(1-q^3) to order 10
# a = q (coeff=1, power=1), n=3
result = aqprod(s, 1, 1, 1, 3, 10)
result

1 - q - q^2 + q^4 + q^5 - q^6 + O(q^10)

This is $(q;q)_3 = (1-q)(1-q^2)(1-q^3)$. Expanding:

$(1-q)(1-q^2) = 1 - q - q^2 + q^3$

$(1 - q - q^2 + q^3)(1-q^3) = 1 - q - q^2 + q^4 + q^5 - q^6$

## Infinite Products

Use `n=None` to compute $(a;q)_\infty = \prod_{k=0}^{\infty}(1-aq^k)$.
Euler's function $(q;q)_\infty = \prod_{k=1}^{\infty}(1-q^k)$ is central
to the theory of partitions.

In [3]:
# Compute (q;q)_inf = prod_{k>=1}(1-q^k) to order 20
euler = aqprod(s, 1, 1, 1, None, 20)
euler

1 - q - q^2 + q^5 + q^7 - q^12 - q^15 + O(q^20)

The sparse pattern $1 - q - q^2 + q^5 + q^7 - q^{12} - q^{15} + \ldots$
reflects **Euler's pentagonal number theorem**. The nonzero exponents
$0, 1, 2, 5, 7, 12, 15, \ldots$ are the generalized pentagonal numbers
$k(3k-1)/2$ for $k = 0, \pm 1, \pm 2, \ldots$

## Partition Generating Function

The **partition function** $p(n)$ counts the number of partitions of $n$.
Its generating function is:

$$\sum_{n=0}^{\infty} p(n)\, q^n = \prod_{k=1}^{\infty} \frac{1}{1 - q^k} = \frac{1}{(q;q)_\infty}$$

In [4]:
from q_kangaroo import partition_gf

pgf = partition_gf(s, 15)
pgf

1 + q + 2*q^2 + 3*q^3 + 5*q^4 + 7*q^5 + 11*q^6 + 15*q^7 + 22*q^8 + 30*q^9 + 42*q^10 + 56*q^11 + 77*q^12 + 101*q^13 + 135*q^14 + O(q^15)

In [5]:
from q_kangaroo import partition_count

for n in range(11):
    print(f"p({n}) = {partition_count(n)}")

p(0) = 1
p(1) = 1
p(2) = 2
p(3) = 3
p(4) = 5
p(5) = 7
p(6) = 11
p(7) = 15
p(8) = 22
p(9) = 30
p(10) = 42


## Named Products -- Eta and Jacobi

q-Kangaroo provides named products from the literature:

- `etaq(s, d, t, order)` computes $\prod_{n \ge 0}(1 - q^{dn+t})$
- `jacprod(s, a, b, order)` computes the Jacobi triple product $J(a, b)$

In [6]:
from q_kangaroo import etaq, jacprod

# etaq(s, 1, 1, 15) computes (q;q)_inf = prod(1-q^n) to order 15
eta1 = etaq(s, 1, 1, 15)
print("eta(q) =", eta1)

# J(1,2) is the Jacobi product equal to theta_4(q)
jp = jacprod(s, 1, 2, 20)
print("J(1,2) =", jp)

eta(q) = 1 - q - q^2 + q^5 + q^7 - q^12 + O(q^15)
J(1,2) = 1 - 2*q + 2*q^4 - 2*q^9 + 2*q^16 + O(q^20)


The Jacobi product $J(1,2)$ equals the theta function
$\theta_4(q) = \sum_{n=-\infty}^{\infty} (-1)^n q^{n^2} = 1 - 2q + 2q^4 - 2q^9 + \ldots$

The nonzero exponents $0, 1, 4, 9, 16$ are perfect squares, with alternating
signs from the $(-1)^n$ factor.

## Discovering Congruences

Ramanujan discovered three remarkable congruences for the partition function:

- $p(5n+4) \equiv 0 \pmod{5}$
- $p(7n+5) \equiv 0 \pmod{7}$
- $p(11n+6) \equiv 0 \pmod{11}$

The `findcong` function discovers such congruences automatically by testing
each residue class for divisibility.

In [7]:
from q_kangaroo import findcong

pgf100 = partition_gf(s, 100)
congs = findcong(pgf100, [5, 7, 11])

for c in congs:
    print(f"p({c['modulus']}n + {c['residue']}) == 0 (mod {c['divisor']})")

p(5n + 4) == 0 (mod 5)
p(7n + 5) == 0 (mod 7)
p(11n + 6) == 0 (mod 11)


All three of Ramanujan's congruences, rediscovered automatically from the
series coefficients alone!

## Your First Identity Verification

q-Kangaroo can **prove** eta-quotient identities using the valence formula
for modular forms. The `prove_eta_id` function takes two eta-quotient
expressions (as lists of `(delta, r_delta)` pairs, where $\eta(\delta\tau)^{r_\delta}$)
and a level $N$, then verifies their equality.

As a demonstration, we verify the self-equality
$\eta(\tau)^2 / \eta(2\tau) = \eta(\tau)^2 / \eta(2\tau)$
at level 2. This trivially-true identity shows the API workflow;
researchers would supply two genuinely different eta-quotient
expressions for real proofs.

In [8]:
from q_kangaroo import prove_eta_id

# Prove eta(tau)^2 / eta(2*tau) = eta(tau)^2 / eta(2*tau) at level 2
result = prove_eta_id(s, [(1, 2), (2, -1)], [(1, 2), (2, -1)], 2)
print(result["status"])

proved


The valence formula confirms the identity is proved!

## Summary

In this notebook we covered the core q-Kangaroo workflow:

| Function | Purpose |
|----------|:--------|
| `QSession()` | Create a computation session |
| `aqprod` | q-Pochhammer symbol $(a;q)_n$ |
| `partition_gf` | Partition generating function $\sum p(n) q^n$ |
| `partition_count` | Compute $p(n)$ directly |
| `etaq` | Generalized eta product $\prod(1-q^{dn+t})$ |
| `jacprod` | Jacobi triple product $J(a,b)$ |
| `findcong` | Automatic congruence discovery |
| `prove_eta_id` | Prove eta-quotient identities via valence formula |

## Next Steps

- **Partition Congruences** (`partition_congruences.ipynb`): Deep dive into
  Ramanujan's congruences with rank and crank generating functions.
- **Series Analysis** (`series_analysis.ipynb`): The full research workflow --
  prodmake, etamake, sift, and relation discovery.
- **Theta Identities** (`theta_identities.ipynb`): Jacobi theta functions,
  triple/quintuple/Winquist products.
- **Hypergeometric Summation** (`hypergeometric_summation.ipynb`): Basic
  hypergeometric series and summation formulas.